In [1]:
#pip install polytope


In [2]:
#pip install pypoman

In [3]:
# pip install polytope
# pip install pypoman

# import math
# import numpy as np
# from sympy.geometry import Point
# import matplotlib.pyplot as plt
# from polytope import polytope
# from polytope.polytope import enumerate_integral_points, qhull, box2poly
# from scipy.spatial import ConvexHull

Let's construct a reflexive 2D polytope (taken from the book "The Calabi–Yau Landscape", page 44)

In [4]:
#V = np.array([[0, -1], [-1, 0], [-1, 2], [1, 0], [1, -1]])


Pypoman provides a form $0\le -Ax+b$

In [5]:


# #print(compute_polytope_vertices(A, b))
# print(ConvexHull(V).npoints)

# GA TEST

In [6]:
# timeout to catch error of qhull, enumerate_integral_points, ABS_TOL infinity loop
# https://stackoverflow.com/questions/21827874/timeout-a-function-windows

from threading import Thread
import functools

#V= np.array([[-4, 3, 1, -3, 2], [1, 4, -4, -2, -3], [-1,  0,  2, -2,  1], [ 4,  0, -3, -2, -4], [ 2, -1,  0, -2,  1], [-1,  2,  0, -2,  1], [ 3, -3,  4,  1, -2]])

def timeout(timeout):
    def deco(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            res = [Exception('function [%s] timeout [%s seconds] exceeded!' % (func.__name__, timeout))]
            def newFunc():
                try:
                    res[0] = func(*args, **kwargs)
                except Exception as e:
                    res[0] = e
            t = Thread(target=newFunc)
            t.daemon = True
            try:
                t.start()
                t.join(timeout)
            except Exception as je:
                print ('error starting thread')
                raise je
            ret = res[0]
            if isinstance(ret, BaseException):
                raise ret
            return ret
        return wrapper
    return deco

In [7]:
import math
import numpy as np
from sympy.geometry import Point
import matplotlib.pyplot as plt
# from polytope import polytope
# from polytope.polytope import enumerate_integral_points, qhull, box2poly
from scipy.spatial import ConvexHull
import random
from random import randint
import copy
from fractions import Fraction
import cdd
from polytope.polytope import qhull, enumerate_integral_points, ABS_TOL
from itertools import combinations
import random

class Population:
    def __init__(self, number_of_generations, survival_rate, crossover_type, crossover_parent_choosing_type, chrom_list, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point):
        #self.generation_0 = Generation(chrom_list, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point)
        #print(True)
        self.number_of_generations = number_of_generations
        self.all_generations = []
        
        self.all_generations.append(Generation(chrom_list, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point))
        #print(True)
        for curr_generation_number in range(0, number_of_generations-1):
            #print(True)
            if curr_generation_number % 10 == 0:
                print("curr Generation count: " + str(curr_generation_number))
            
            new_gen_list = []
            if curr_generation_number % (number_of_generations/4) == 0 and curr_generation_number != 0:
                copy_prev_Generation_chrom_list = copy.deepcopy(self.all_generations[curr_generation_number].get_chrom_list())

                new_gen_list.append(copy.deepcopy(copy_prev_Generation_chrom_list[0]))
                #new_gen_list.append(copy.deepcopy(copy_prev_Generation_chrom_list[0]))
                for i in range(0, len(copy_prev_Generation_chrom_list)-1):
                    if copy_prev_Generation_chrom_list[i].get_chromosom_string() != copy_prev_Generation_chrom_list[i+1].get_chromosom_string():
                        new_gen_list.append(copy.deepcopy(copy_prev_Generation_chrom_list[i+1]))


                
                while len(new_gen_list) < num_chroms_per_Generation:
                    #chromosome_test_points=[]
                    polytope_points = []

                    for i in range(0,num_points_per_polytope):
                        polytope_points.append(random.sample(range(min_value_per_point, max_value_per_point), num_dimensions))
                    
                    #polytope_points.sort(key=lambda x: int(x[0]))###########################################################################################
                    
                    chromosome_polytope_points = np.array(polytope_points)
                    #print(polytope_points)
                    chromosom_test = Chromosom(chromosome_polytope_points)

                    if chromosom_test.get_is_valid():
                        new_gen_list.append(chromosom_test)
                
                new_gen_list.sort(key=lambda x: x.fitness, reverse=True)
                
                print("REDUCTION PERFORMED!!!")
                
                #XXXXXX     .sort(key=lambda x: x.fitness, reverse=True)
                self.all_generations.append(Generation(new_gen_list, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point))


            else:    
                copy_prev_Generation_chrom_list = copy.deepcopy(self.all_generations[curr_generation_number].get_chrom_list())
                copy_prev_Generation_chrom_list = copy_prev_Generation_chrom_list[:int(len(copy_prev_Generation_chrom_list)/2)]

                #parents_pool = list(combinations(copy_prev_Generation_chrom_list, 2))
                




                parents = []
                while len(new_gen_list)+len(copy_prev_Generation_chrom_list) < num_chroms_per_Generation: #len(self.all_generations[0].get_chrom_list()):
                    parents = random.sample(copy_prev_Generation_chrom_list, 2)
                    
                    crossover_chromosom = Chromosom.apply_crossover(parents[0], parents[1], crossover_type)
                    
                    if crossover_chromosom.get_is_valid():
                        new_gen_list.append(crossover_chromosom)
                    
                
                
                copy_prev_Generation_chrom_list.extend(new_gen_list)
                copy_prev_Generation_chrom_list.sort(key=lambda x: x.fitness, reverse=True)
                self.all_generations.append(Generation(copy_prev_Generation_chrom_list, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point))


    def get_all_generations(self):
        return self.all_generations


class Generation:
    num_points_per_polytope = 0
    num_chroms_per_Generation = 0
    num_dimensions = 0 # 3
    min_value_per_point = 0  # -3
    max_value_per_point = 0  # 3
    
    
    def __init__(self, chrom_list, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point):
        
        Generation.num_points_per_polytope = num_points_per_polytope
        Generation.num_chroms_per_Generation = num_chroms_per_Generation
        Generation.num_dimensions = num_dimensions # 3
        Generation.min_value_per_point = min_value_per_point  # -3
        Generation.max_value_per_point = max_value_per_point  # 3
        print(1)
        if chrom_list == None:

            self.chrom_list = self.gen_first_generation(num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point)
        else:
            self.chrom_list = chrom_list
        print("two")
        self.generation_fitness_list = self.set_fitness_value_list(self.chrom_list)

    def set_fitness_value_list(self, chrom_list):
        fitness = []
        for chrom in chrom_list:
            fitness.append(chrom.get_fitness())

        #print(fitness)
        return(fitness)

    def get_chrom_list(self):
        return self.chrom_list
    
    def get_dimension():
        return Generation.num_dimensions

    def get_min_value_per_point():
        return Generation.min_value_per_point
    
    def get_max_value_per_point():
        return Generation.max_value_per_point

    def get_fitness_values_list(self):
        return self.generation_fitness_list

    def gen_first_generation(self, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point):
        #num_points_per_polytope = 5
        #num_chroms_per_Generation = 10
        #num_dimensions = num_dimensions

        polytope_points = []
        generation_polytopes = []
        counter = 0
        #for j in range(0, num_chroms_per_Generation):
        while (len(generation_polytopes)+len(polytope_points)) < num_chroms_per_Generation:
            #chromosome_test_points=[]
            print(len(generation_polytopes)+len(polytope_points))
            polytope_points = []
            
            # if len(generation_polytopes) == 0:
            #     polytope_points = [[-2, 3, 1, -3, 0], [1,-2,-1, 2, 0], [ 4,-4, 1, 2,-2], [ 1,2,3,0,-1], [-3,3,-1,-2,2], [ 1,2,3,-3,-1], [-2,-1,-3,0,2]]
            
            # elif(len(generation_polytopes) == 1):
            #     polytope_points = [[4, -2,  3,  2, -3], [ 1, -2, -1,  2,  0], [ 4, -4,  1,  2, -2], [ 1,  2,  3,  0, -1], [-3,  3, -1, -2,  2], [ 1,  2,  3, -3, -1,], [-2, -4, -5,  2,  1]]

            # else:
                
            #print(counter)
            
            counter += 1
            for i in range(0,num_points_per_polytope):
                polytope_points.append(random.sample(range(min_value_per_point, max_value_per_point), num_dimensions))

            #print(polytope_points)
                
            #polytope_points.sort(key=lambda x: int(x[0])) ################################################################################
            
            chromosome_polytope_points = np.array(polytope_points)
            #print(polytope_points)
            chromosom_test = Chromosom(chromosome_polytope_points)
            #print(chromosom_test.get_is_valid())
            if chromosom_test.get_is_valid():
                generation_polytopes.append(chromosom_test)
        
        generation_polytopes.sort(key=lambda x: x.fitness, reverse=True)
        #print("generation done")
        return generation_polytopes

class Chromosom:
    chrom_count = 0
    reflexive_politopes = {}
    all_chromosomes = {}

    def __init__(self, points):
        chromosome_polytope_points = points
        #print("pre try")
        #print(Generation.num_dimensions)
        self.is_valid = False
        self.chromosom_string = np.array2string(chromosome_polytope_points).replace("\n", ",")

        if self.chromosom_string in Chromosom.all_chromosomes.keys():
            self.fitness = Chromosom.all_chromosomes[self.chromosom_string]
            Chromosom.chrom_count = Chromosom.chrom_count + 1
            self.gen_list = chromosome_polytope_points
            self.is_valid = True
        else:
            #result = None
            #while result is None:
            try:
                internal_points = []
                #print("###############################################")
                Chromosom.chrom_count = Chromosom.chrom_count + 1
                #print(Chromosom.chrom_count)
                
                self.gen_list = chromosome_polytope_points
                self.fitness = self.calc_fitness(chromosome_polytope_points)
                #print(True)
                self.is_valid = True


                if self.fitness == 0 and self.chromosom_string not in Chromosom.reflexive_politopes.keys(): 
                    print(True)
                    if np.min(self.gen_list) < 0:
                        self.fitness = np.max(self.gen_list) + (np.min(self.gen_list) *-1)
                    elif np.min(self.gen_list) > 0:
                        self.fitness = np.max(self.gen_list) - np.min(self.gen_list)   
                    elif np.min(self.gen_list)==0:
                        self.fitness = np.max(self.gen_list)
                    polytope_volume = ConvexHull(chromosome_polytope_points).volume #qhull(chromosome_polytope_points).volume
                    Chromosom.reflexive_politopes[polytope_volume] = [ConvexHull(self.gen_list), self.gen_list]
                    Chromosom.reflexive_politopes[self.chromosom_string] = [repr(ConvexHull(self.gen_list)), self.gen_list]
                    #print(Chromosom.reflexive_politopes.keys())
                        
                    Chromosom.all_chromosomes[self.chromosom_string] = self.fitness

                #result = 1
            except:
                self.is_valid = False
    
    def get_chromosom_string(self):
        return self.chromosom_string

    def get_is_valid(self):
        return self.is_valid
    
    def get_gen_list(self):
        return self.gen_list
    
    def get_fitness(self):
        return self.fitness

    def get_gen_list(self):
        return self.gen_list

    #code based on https://github.com/stephane-caron/pypoman/blob/master/pypoman/duality.py
    def compute_distances(self, vertices: np.ndarray) -> np.ndarray:
        #print("yyyyyyyyyyyyyyyyyyyyyyyyyyyyy")
        
        V = np.vstack(vertices)
        t = np.ones((V.shape[0], 1))  # first column is 1 for vertices
        tV = np.hstack([t, V])
        mat = cdd.Matrix(tV, number_type="fraction")
        mat.rep_type = cdd.RepType.GENERATOR
        P = cdd.Polyhedron(mat)
        bA = np.array(P.get_inequalities())
        if bA.shape == (0,):  # bA == []
            return bA
        # the polyhedron is given by b + A x >= 0 where bA = [b|A]
        b = bA[:, 0]
        A = -np.array(bA[:, 1:])

        for i, row in enumerate(A):
            denominators = []
            has_fraction = False
            for j in row:
                if type(j) == Fraction:
                    has_fraction = True
                    denominators.append(j.denominator)    
            if has_fraction:
                b[i] *= np.lcm.reduce(denominators)
        return b

    @timeout(10)
    def calc_fitness(self, vertices):
        
        # integral_points = enumerate_integral_points(qhull(vertices))
        # integral_points = integral_points.transpose()
        # integral_points = integral_points.astype(int)
        # ip_count = len(integral_points)
        # print("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")
        # print(np.array(vertices))
        distances = self.compute_distances(np.array(vertices))
        result = 0
        # if ip_count > 1:
        #     result -= 1
        for d in distances:
            result -= abs(d-1)
        
        return result

    def apply_crossover(chromosom_1, chromosom_2, crossover_type):
        #print("crossover")
        # chromosom_1 =
        # chromosom_2
        crossover_type = crossover_type

        new_chromosom_gen_list = []
        if crossover_type == 1: ##############to be removed

            if chromosom_1.get_fitness() > chromosom_2.get_fitness():
                chromosom_1_points = chromosom_1.get_gen_list()
                chromosom_2_points = chromosom_2.get_gen_list()
            else:
                chromosom_2_points = chromosom_1.get_gen_list()
                chromosom_1_points = chromosom_2.get_gen_list()

            
            if len(chromosom_1_points)>len(chromosom_2_points):
                length_new_chromosom = len(chromosom_2_points)
                random_crossover_point = random.randint(1,len(chromosom_2_points)-1)
            else:
                length_new_chromosom = len(chromosom_2_points)
                random_crossover_point = random.randint(1,len(chromosom_2_points)-1)

            #new_gen = []
            for i in range(0,length_new_chromosom):
                
                if random.random() < 0.8:
                    if i < random_crossover_point:
                        new_gen = []
                        new_gen_2 = []
                        for j in chromosom_1_points[i]:
                            new_gen.append(j)#[chromosom_1_points[0],chromosom_1_points[1]])

                    else:
                        new_gen = []
                        for j in chromosom_2_points[i]:
                            new_gen.append(j)

                    new_chromosom_gen_list.append(new_gen)
                    #print("random_gen_test")
                else:
                    new_chromosom_gen_list.append(random.sample(range(Generation.min_value_per_point, Generation.max_value_per_point), Generation.num_dimensions))
                    #print("random_gen_test")

        #new_chromosom_gen_list.sort(key=lambda x: int(x[0]))###################################################################################################################

            new_chromosom = Chromosom(np.array(new_chromosom_gen_list))

            return new_chromosom





    #def apply_crossover_return_2_children(chromosom_1, chromosom_2, crossover_type):



In [10]:
# Population Test:

# test_population = Population(number_of_generations, survival_rate, crossover_type, crossover_parent_choosing_type, chrom_list, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point)
number_of_generations = 1000
survival_rate = None
crossover_type = 1
crossover_parent_choosing_type = None
chrom_list = None
num_points_per_polytope = 7
num_chroms_per_Generation = 30
num_dimensions = 5
min_value_per_point = -15
max_value_per_point = 15

test_population = Population(number_of_generations, survival_rate, crossover_type, crossover_parent_choosing_type, chrom_list, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point)



True
1
0
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
two
True
True
curr Generation count: 0
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 10
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 20
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 30
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 40
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 50
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 60
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two


In [11]:

for i in Chromosom.reflexive_politopes:
    print(i)
    print(Chromosom.reflexive_politopes[i][1])

print(Chromosom.reflexive_politopes)
print(len(Chromosom.reflexive_politopes))

{}
0


In [12]:
# V = np.array([[1, 0, -2], [-2, -3, 2], [-1, -2, 1], [1, 2, -1], [1, 2, 0]])

# print(Chromosom.calc_fitness(V))

In [13]:
test_population_generations = test_population.get_all_generations()
test_population_generations_chroms = test_population_generations[1].get_chrom_list()
#print(test_population_generations_chroms)

all_fitness = []
for generation in test_population.get_all_generations():
    #fitness = [generation.get_fitness_values_list()]
    # for chrom in generation.get_chrom_list():
    #     fitness.append(chrom.get_fitness())

    print(generation.get_fitness_values_list())

    all_fitness.append(generation.get_fitness_values_list())
    #print(generation.get_fitness_values_list())
    #all_fitness.append(fitness)
#print(all_fitness)
print(len(all_fitness))
#print(all_fitness)
print(Chromosom.reflexive_politopes)
print(len(Chromosom.reflexive_politopes))

[Fraction(-319262, 1), Fraction(-342794, 1), Fraction(-444590, 1), Fraction(-677114, 1), Fraction(-818439, 1), Fraction(-863964, 1), Fraction(-875220, 1), Fraction(-1208994, 1), Fraction(-1318202, 1), Fraction(-1636455, 1), Fraction(-1959494, 1), Fraction(-2320407, 1), Fraction(-2459220, 1), Fraction(-2776361, 1), Fraction(-3510751, 1), Fraction(-3920811, 1), Fraction(-4558711, 1), Fraction(-4816187, 1), Fraction(-4988494, 1), Fraction(-5507509, 1), Fraction(-5887397, 1), Fraction(-7357795, 1), Fraction(-7779174, 1)]
[Fraction(-310259, 1), Fraction(-319262, 1), Fraction(-342794, 1), Fraction(-444590, 1), Fraction(-645012, 1), Fraction(-677114, 1), Fraction(-818439, 1), Fraction(-863964, 1), Fraction(-875220, 1), Fraction(-876641, 1), Fraction(-948348, 1), Fraction(-1208994, 1), Fraction(-1278033, 1), Fraction(-1310891, 1), Fraction(-1318202, 1), Fraction(-1344460, 1), Fraction(-1438451, 1), Fraction(-1619268, 1), Fraction(-1636455, 1), Fraction(-1865914, 1), Fraction(-1947093, 1), Frac

In [38]:
# test_population_generations = test_population.get_all_generations()
# test_population_generations_chroms = test_population_generations[2].get_chrom_list()
# #print(test_population_generations_chroms)

# print(test_population_generations_chroms[0].get_gen_list())
# print(test_population_generations_chroms[0].get_fitness())

# # all_fitness = []
# # for generation in test_population.get_all_generations():
# #     fitness = [generation.get_fitness_values_list()]
# #     for chrom in generation.get_chrom_list():
# #         fitness.append(chrom.get_fitness())

# #     #print(generation.get_fitness_values_list())

# #     all_fitness.append(generation.get_fitness_values_list())
# #     #print(generation.get_fitness_values_list())
# #     #all_fitness.append(fitness)
# # #print(all_fitness)
# # print(len(all_fitness))
# # #print(all_fitness)

In [14]:
chrom_list = test_population_generations[len(test_population.get_all_generations())-1].get_chrom_list()

test_population2 = Population(number_of_generations, survival_rate, crossover_type, crossover_parent_choosing_type, chrom_list, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point)

True
1
two
True
True
curr Generation count: 0
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 10
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 20
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 30
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 40
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 50
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 60
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 70
1


In [17]:
test_population_generations2 = test_population2.get_all_generations()
test_population_generations_chroms2 = test_population_generations2[1].get_chrom_list()
#print(test_population_generations_chroms)

all_fitness = []
for generation in test_population2.get_all_generations():
    #fitness = [generation.get_fitness_values_list()]
    # for chrom in generation.get_chrom_list():
    #     fitness.append(chrom.get_fitness())

    print(generation.get_fitness_values_list())

    all_fitness.append(generation.get_fitness_values_list())
    #print(generation.get_fitness_values_list())
    #all_fitness.append(fitness)
#print(all_fitness)
print(len(all_fitness))
#print(all_fitness)
print(Chromosom.reflexive_politopes)
print(len(Chromosom.reflexive_politopes))

[Fraction(-148, 1), Fraction(-148, 1), Fraction(-148, 1), Fraction(-148, 1), Fraction(-148, 1), Fraction(-148, 1), Fraction(-148, 1), Fraction(-148, 1), Fraction(-148, 1), Fraction(-148, 1), Fraction(-148, 1), Fraction(-148, 1), Fraction(-148, 1), Fraction(-148, 1), Fraction(-148, 1), Fraction(-148, 1), Fraction(-148, 1), Fraction(-148, 1), Fraction(-754, 1), Fraction(-2501, 1), Fraction(-3186, 1), Fraction(-3284, 1), Fraction(-24064, 1), Fraction(-26822, 1), Fraction(-31411, 1), Fraction(-145778, 1), Fraction(-206054, 1), Fraction(-242503, 1), Fraction(-254233, 1), Fraction(-387556, 1)]
[Fraction(-148, 1), Fraction(-148, 1), Fraction(-148, 1), Fraction(-148, 1), Fraction(-148, 1), Fraction(-148, 1), Fraction(-148, 1), Fraction(-148, 1), Fraction(-148, 1), Fraction(-148, 1), Fraction(-148, 1), Fraction(-148, 1), Fraction(-148, 1), Fraction(-148, 1), Fraction(-148, 1), Fraction(-148, 1), Fraction(-148, 1), Fraction(-148, 1), Fraction(-922, 1), Fraction(-1605, 1), Fraction(-3256, 1), Fra

In [16]:
chrom_list = test_population_generations2[len(test_population2.get_all_generations())-1].get_chrom_list()

test_population3 = Population(number_of_generations, survival_rate, crossover_type, crossover_parent_choosing_type, chrom_list, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point)

True
1
two
True
True
curr Generation count: 0
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 10
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 20
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 30
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 40
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 50
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 60
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 70
1


In [18]:
test_population_generations3 = test_population3.get_all_generations()
#test_population_generations_chroms2 = test_population_generations2[999].get_chrom_list()
#print(test_population_generations_chroms)

all_fitness = []
for generation in test_population3.get_all_generations():
    #fitness = [generation.get_fitness_values_list()]
    # for chrom in generation.get_chrom_list():
    #     fitness.append(chrom.get_fitness())

    print(generation.get_fitness_values_list())

    all_fitness.append(generation.get_fitness_values_list())
    #print(generation.get_fitness_values_list())
    #all_fitness.append(fitness)
#print(all_fitness)
print(len(all_fitness))
#print(all_fitness)
print(Chromosom.reflexive_politopes)
print(len(Chromosom.reflexive_politopes))

[-102, -102, -102, -102, -102, -102, -102, -102, -102, -102, -102, -102, -102, -102, -102, -102, -102, Fraction(-35075, 1), Fraction(-37917, 1), Fraction(-83638, 1), Fraction(-114477, 1), Fraction(-121580, 1), Fraction(-125370, 1), Fraction(-138108, 1), Fraction(-177126, 1), Fraction(-210993, 1), Fraction(-315044, 1), Fraction(-398517, 1), Fraction(-401717, 1), Fraction(-1475329, 1)]
[-102, -102, -102, -102, -102, -102, -102, -102, -102, -102, -102, -102, -102, -102, -102, -102, -102, Fraction(-28948, 1), Fraction(-52422, 1), Fraction(-58450, 1), Fraction(-81500, 1), Fraction(-103659, 1), Fraction(-169377, 1), Fraction(-185829, 1), Fraction(-194510, 1), Fraction(-222424, 1), Fraction(-273913, 1), Fraction(-514206, 1), Fraction(-797369, 1), Fraction(-2094856, 1)]
[-102, -102, -102, -102, -102, -102, -102, -102, -102, -102, -102, -102, -102, -102, -102, -102, Fraction(-13142, 1), Fraction(-27439, 1), Fraction(-58662, 1), Fraction(-63046, 1), Fraction(-70537, 1), Fraction(-70934, 1), Frac

In [19]:
chrom_list = test_population_generations3[len(test_population3.get_all_generations())-1].get_chrom_list()

test_population4 = Population(number_of_generations, survival_rate, crossover_type, crossover_parent_choosing_type, chrom_list, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point)

True
1
two
True
True
curr Generation count: 0
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 10
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 20
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 30
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 40
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 50
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 60
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 70
1


In [20]:
test_population_generations4 = test_population4.get_all_generations()
#test_population_generations_chroms2 = test_population_generations2[999].get_chrom_list()
#print(test_population_generations_chroms)

all_fitness = []
for generation in test_population4.get_all_generations():
    #fitness = [generation.get_fitness_values_list()]
    # for chrom in generation.get_chrom_list():
    #     fitness.append(chrom.get_fitness())

    print(generation.get_fitness_values_list())

    all_fitness.append(generation.get_fitness_values_list())
    #print(generation.get_fitness_values_list())
    #all_fitness.append(fitness)
#print(all_fitness)
print(len(all_fitness))

print(Chromosom.reflexive_politopes)
print(len(Chromosom.reflexive_politopes))

[-53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, Fraction(-8175, 1), Fraction(-11244, 1), Fraction(-20500, 1), Fraction(-24284, 1), Fraction(-39373, 1), Fraction(-98062, 1), Fraction(-98748, 1), Fraction(-261180, 1), Fraction(-326966, 1), Fraction(-383658, 1), Fraction(-509895, 1), Fraction(-926971, 1), Fraction(-1481879, 1)]
[-53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, Fraction(-15716, 1), Fraction(-41601, 1), Fraction(-49012, 1), Fraction(-51536, 1), Fraction(-152533, 1), Fraction(-161567, 1), Fraction(-188106, 1), Fraction(-214359, 1), Fraction(-349732, 1), Fraction(-370221, 1), Fraction(-394026, 1), Fraction(-481351, 1), Fraction(-1898062, 1)]
[-53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, Fraction(-31075, 1), Fraction(-65940, 1), Fraction(-85559, 1), Fraction(-134085, 1), Fraction(-135008, 1), Fraction(-139483, 1), Fraction(-191602, 1), Fraction(-201505, 1), Fraction(-33

In [21]:
chrom_list = test_population_generations4[len(test_population4.get_all_generations())-1].get_chrom_list()

#number_of_generations = 200


test_population5 = Population(number_of_generations, survival_rate, crossover_type, crossover_parent_choosing_type, chrom_list, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point)



True
1
two
True
True
curr Generation count: 0
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 10
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 20
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 30
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 40
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 50
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 60
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
1
two
True
curr Generation count: 70
1


In [22]:
test_population_generations5 = test_population5.get_all_generations()
#test_population_generations_chroms2 = test_population_generations2[999].get_chrom_list()
#print(test_population_generations_chroms)

all_fitness = []
for generation in test_population4.get_all_generations():
    #fitness = [generation.get_fitness_values_list()]
    # for chrom in generation.get_chrom_list():
    #     fitness.append(chrom.get_fitness())

    print(generation.get_fitness_values_list())

    all_fitness.append(generation.get_fitness_values_list())
    #print(generation.get_fitness_values_list())
    #all_fitness.append(fitness)
#print(all_fitness)
print(len(all_fitness))
print(Chromosom.reflexive_politopes)
print(len(Chromosom.reflexive_politopes))

[-53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, Fraction(-8175, 1), Fraction(-11244, 1), Fraction(-20500, 1), Fraction(-24284, 1), Fraction(-39373, 1), Fraction(-98062, 1), Fraction(-98748, 1), Fraction(-261180, 1), Fraction(-326966, 1), Fraction(-383658, 1), Fraction(-509895, 1), Fraction(-926971, 1), Fraction(-1481879, 1)]
[-53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, Fraction(-15716, 1), Fraction(-41601, 1), Fraction(-49012, 1), Fraction(-51536, 1), Fraction(-152533, 1), Fraction(-161567, 1), Fraction(-188106, 1), Fraction(-214359, 1), Fraction(-349732, 1), Fraction(-370221, 1), Fraction(-394026, 1), Fraction(-481351, 1), Fraction(-1898062, 1)]
[-53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, -53, Fraction(-31075, 1), Fraction(-65940, 1), Fraction(-85559, 1), Fraction(-134085, 1), Fraction(-135008, 1), Fraction(-139483, 1), Fraction(-191602, 1), Fraction(-201505, 1), Fraction(-33

In [23]:
# a = [1,2,3]
# b = a.copy()
# #b = []
# #b.append(a[0])
# #b.append(a[1])
# a[1] = 5

# print(a)
# print(b)

for i in Chromosom.reflexive_politopes:
    print(i)
    print(Chromosom.reflexive_politopes[i][1])

print(Chromosom.reflexive_politopes)
print(len(Chromosom.reflexive_politopes))

{}
0


In [48]:
# Crossover test - to be checked

# # #def apply_crossover(chromosom_1, chromosom_2, crossover_type):
# chromosom_1_points = test_polytopes[1].get_gen_list()
# chromosom_2_points = test_polytopes[0].get_gen_list()
# # crossover_type = 1

# # new_chromosom_gen_list = []
# # if crossover_type == 1:
    

# #     if len(chromosom_1_points)>len(chromosom_2_points):
# #         length_new_chromosom = len(chromosom_2_points)
# #         random_crossover_point = random.randint(1,len(chromosom_2_points)-1)
# #     else:
# #         length_new_chromosom = len(chromosom_2_points)
# #         random_crossover_point = random.randint(1,len(chromosom_2_points)-1)

# #     #new_gen = []
# #     for i in range(0,length_new_chromosom):
# #         if i <= random_crossover_point:
# #             new_gen = []
# #             for j in chromosom_1_points[i]:
# #                 new_gen.append(j)#[chromosom_1_points[0],chromosom_1_points[1]])
# #         else:
# #             new_gen = []
# #             for j in chromosom_2_points[i]:
# #                 new_gen.append(j)

# #         new_chromosom_gen_list.append(new_gen)

# # new_chromosom = Chromosom(np.array(new_chromosom_gen_list))
# #return new_chromosom

# new_chromosom = Chromosom.apply_crossover(test_polytopes[1], test_polytopes[0], 1)

# #print(random_crossover_point)
# print(chromosom_1_points)
# print(chromosom_2_points)
# print(new_chromosom.get_gen_list())
# print(new_chromosom.get_fitness())


In [49]:
# import numpy as np
# import polytope

# # halfspace representation computed using `polytope`
# # from the vertex representation given in the question
# vertices = np.array([[0, -1], [-1, 0], [-1, 2], [1, 0], [1, -1]])
# poly = polytope.qhull(vertices)
# # first halfspace representation
# A = np.array([
#     [0, -1],
#     [-1, 0],
#     [-1, -1],
#     [1, 1],
#     [1, 0]])
# b = np.array([1, 1, 1, 1, 1])
# question_poly_1 = polytope.Polytope(A, b)
# # second halfspace representation
# A = np.array([
#     [-0.70711, -0.70711],
#     [-1, -0],
#     [0.70711, 0.70711],
#     [0, -1],
#     [1, 0]])
# b = np.array([0.70711, 1, 0.70711, 1, 1])

# question_poly_2 = polytope.Polytope(A, b)
# # check that all the above halfspace representations
# # represent the same polytope
# assert poly == question_poly_1, (poly, question_poly_1)
# assert poly == question_poly_2, (poly, question_poly_2)

In [50]:
# inner_point = np.array([0,0]) # in this case it is the origin
# fac1 = polytope.quickhull.Facet([[0, -1], [-1, 0]])
# fac2 = polytope.quickhull.Facet([[-1, 2], [1, 0]])
# fac3 = polytope.quickhull.Facet([[1, -1], [0, -1]])

# d1 = polytope.quickhull.distance(inner_point, fac1)
# d2 = polytope.quickhull.distance(inner_point, fac2)
# d3 = polytope.quickhull.distance(inner_point, fac3)
# print(d1)
# print(d2)
# print(d3)

Let's visualize our 2D polytope. For this purpose we use the [Polytope Library](https://github.com/tulip-control/polytope)

In [51]:
# project into 2D



In [52]:

# print(polytope.polytope.cheby_ball(P))

# min_V = np.min(V, axis=0) #min val of each cols
# min_x = min_V[0]
# min_y = min_V[1]
# max_V = np.max(V, axis=0) #max val of each cols
# max_x = max_V[0]
# max_y = max_V[1]

# ax = P.plot(linestyle="solid", linewidth=1, color="lightblue", edgecolor="red")
# ax.set_xlim([min_x-1, max_x+1])
# ax.set_ylim([min_y-1, max_y+1])
# plt.show()